In [1]:
import pandas as pd
import shutil
from os import listdir, mkdir, path
from pathlib import Path
import sys
from email.parser import HeaderParser
import collections
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import time
from collections import Counter

In [6]:
parser = HeaderParser()

with open('features.txt', 'r') as f:
    columns = [line.strip() for line in f.readlines()]
list_of_rows = []

def getIndexMap(index_path, data_path):
    index = {}
    with open(index_path, encoding='us-ascii') as index_file:
        for i, line in enumerate(index_file):
            type = line[0:4]
            file_path = line.split('/')[2][:-1]
            if type == 'spam':
                index[f'{data_path}{file_path}'] = 1
            else:
                index[f'{data_path}{file_path}'] = 0
    return index


def addEmailToList(file_path, index):
    global parser, list_of_rows
    
    # The label for the email: ham or spam
    label = index[file_path]
    hops = 0
    emailStr = ""
    row_dict = {}
    
    # Read the full email content
    try:
        with open(file_path, encoding='latin_1') as emailFile:
            for line in emailFile:
                emailStr += line
    except UnicodeDecodeError:
        pass
    
    # Parse the email content
    h = parser.parsestr(emailStr)
    
    # Parse recieved field
    received_list = h.get_all('received')
    hops = 0
    if received_list is not None:
        hops = len(received_list)
        col_name_recieved = 'received'
        
        for inx, received_field in enumerate(received_list):
            col = col_name_recieved + str(inx+1)
            row_dict[col] = received_field

    
    # Make everything lowercase to avoid issues
    features_lower_case = [x.lower() for x in h.keys()]
    
    # Parse everything else
    new_row = dict(zip(features_lower_case, h.values()))
    new_row['hops'] = hops
    new_row['body'] = h.get_payload(decode=True)
    
    
    for key,value in new_row.items():
        if key in columns:
            row_dict['label'] = label
            row_dict[key] = value
    
    
    list_of_rows.append(row_dict)

def main(directory):
    global list_of_rows
    index = getIndexMap(directory + 'full/index', directory + 'data/')
    start_time = time.time()
    counter = 0
    # Go through each email in the data set
    for emailFile in listdir(directory + 'data'):
        path = directory + 'data/' + emailFile
        addEmailToList(path, index)
    
    # Create the dataframe
    df = pd.DataFrame(list_of_rows)
    df = df[columns]
    
    # Output the dataframe to a .csv file
    df.to_csv(directory + 'trec07p.csv', index=False)
    print('dataset was built at: ' + directory + 'trec07p.csv' )

directory = '../datasets/trec07p/'
main(directory)

dataset was built at: ../datasets/trec07p/trec07p.csv
